### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [1]:
include("jupy.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


main (generic function with 1 method)

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

In [22]:
using Cbc
using JuMP
function solveGoogle(data::DataGoogle, verbose::Bool)

    # model
    m = Model(Cbc.Optimizer)    # on crée un modèle en précisant qu'on va utiliser Cbc
    
    
    @variable(m,x[i=1:data.nbMachines,j=1:data.nbProcess],Bin) # Process j affecté à machine i (xij is Binary)

    println("---------Nbre machines----------")
    println(data.nbMachines)
    println("---------Nbre processus---------")
    println(data.nbProcess)    
    println("--------------------------------")
 
    ####################### O3 ########################
    #@objective(m, Min, sum(data.processMoveCost[j] * ( 1 - x[data.initialAssignment[j],j] ) for j in 1:data.nbProcess)) #O3
    ####################################################
    
    
   ####################### Contrainte 1 ######################################
    @constraint(m, constraint1[j=1:data.nbProcess], sum(x[i,j] for i=1:data.nbMachines) == 1 ) # C1
    ###########################################################################
    
    
   ####################### Contrainte 2 ######################################
    @constraint(m, constraint2[i=1:data.nbMachines,r=1:data.nbResources], sum(data.processReq[j,r]*(x[i,j]) for j=1:data.nbProcess) <= data.hardCapacities[i,r] ) # C2
    ###########################################################################
    
    
   ####################### Contrainte 3 ######################################
    for p1 in 1:data.nbProcess 
        for p2 in 1:data.nbProcess
            if data.servicesProcess[p1]==data.servicesProcess[p2]
                if p1 != p2
                    for i in 1:data.nbMachines
                        @constraint(m, x[i,p1]+x[i,p2] <= 1 )
                    end
                end
            end
        end
    end   
    ###########################################################################
      
    
    ####################### O1 ########################
    @variable(m,max_value_o1[i=1:data.nbMachines,r=1:data.nbResources] >= 0,Int) # to set max
    @variable(m,is_max_value_o1[i=1:data.nbMachines,r=1:data.nbResources],Bin) # to help set max

    @constraint(m,init_max_value_o1[i=1:data.nbMachines,r=1:data.nbResources],max_value_o1[i,r] >= sum(data.processReq[p,r] * x[i,p] for p in 1:data.nbProcess) - data.softCapacities[i,r])
   
    @constraint(m,check1_init_max_value_o1[i=1:data.nbMachines,r=1:data.nbResources],max_value_o1[i,r]<= 0 + 10^10 * is_max_value_o1[i,r])
    
    @constraint(m,check2_init_max_value_o1[i=1:data.nbMachines,r=1:data.nbResources],max_value_o1[i,r] <= sum(data.processReq[p,r] * x[i,p] for p in 1:data.nbProcess) - data.softCapacities[i,r] + 10^10 * (1 - is_max_value_o1[i,r]) )    
    
    #@objective(m,Min, sum(data.weightLoadCost[r] * max_value_o1[i,r] for i in 1:data.nbMachines, r in 1:data.nbResources))

    ####################################################
    
    ####################### O2 ########################


    @variable(m, max_1[i=1:data.nbMachines,j=1:data.nbBalanceCostData] >= 0, Int) # to set max
    @variable(m,is_max1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],Bin) # to help set max


    @constraint(m,init_max1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],max_1[i,j] >= data.balanceCostDataList[j].target * ( data.hardCapacities[i,data.balanceCostDataList[j].resource1] - sum(data.processReq[p,data.balanceCostDataList[j].resource1] * x[i,p] for p in 1:data.nbProcess) ) - ( data.hardCapacities[i,data.balanceCostDataList[j].resource2] - sum(data.processReq[p ,data.balanceCostDataList[j].resource2] * x[i,p] for p in 1:data.nbProcess) ) )
    
    @constraint(m,chack1_int_1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],max_1[i,j] <= 0 + 10^10 * is_max1[i,j] )

    @constraint(m,chack2_int_1[i=1:data.nbMachines,j=1:data.nbBalanceCostData],max_1[i,j] <= data.balanceCostDataList[j].target * ( data.hardCapacities[i,data.balanceCostDataList[j].resource1] - sum(data.processReq[p,data.balanceCostDataList[j].resource1] * x[i,p] for p in 1:data.nbProcess) ) - ( data.hardCapacities[i,data.balanceCostDataList[j].resource2] - sum(data.processReq[p ,data.balanceCostDataList[j].resource2] * x[i,p] for p in 1:data.nbProcess) ) + 10^10 * (1 - is_max1[i,j]))
    
    
    #@objective(m,Min, sum(data.balanceCostDataList[j].weight * max_1[m,j] for m in 1:data.nbMachines, j in 1:data.nbBalanceCostData))
    
    ####################################################
    
    ####################### O5 ########################
    
    #@objective(m,Min,sum(data.machineMoveCosts[data.initialAssignment[p],m]*x[m,p]  for p in 1:data.nbProcess,m in 1:data.nbMachines))

    ####################################################

    ####################### O4 ########################
    @variable(m,nbMovesInService[i=1:data.nbServices] >= 0,Int) #tableau de valeurs des compteurs pour les services
    @constraint(m,init_nbMovesInService[i=1:data.nbServices],nbMovesInService[i] == sum((1 - x[data.initialAssignment[p],p]) for p in 1:data.nbProcess if data.servicesProcess[p] == i)) #initialise ce tableau
    
    @variable(m,max_move_cervices,Int)
    @constraint(m,init_max_move_cervices[i=1:data.nbServices],max_move_cervices >= nbMovesInService[i])
    
    #@objective(m,Min,max_move_cervices) # minimiser 
    ####################### C4 ###########################
    
    for s1 in 1:data.nbServices
        for s2 in 1:data.nbServices
            if s1 != s2
                for pa in data.dependeces[s1]
                    @constraint(m,constrain5[pb in data.dependeces[s2], n in 1:data.nbMachines],sum(data.machineMoveCosts[data.initialAssignment[p],m]*x[m,p] for p in 1:data.nbProcess) == m) )
                        
            

    
    ####################### O TOTAL ####################
    
    @objective(m,Min,max_move_cervices*data.serviceMoveWeight+sum(data.machineMoveCosts[data.initialAssignment[p],m]*x[m,p] for p in 1:data.nbProcess,m in 1:data.nbMachines)*data.machineMoveWeight + data.processMoveWeight*sum(data.processMoveCost[j] * ( 1 - x[data.initialAssignment[j],j] ) for j in 1:data.nbProcess)+sum(data.balanceCostDataList[j].weight * max_1[m,j] for m in 1:data.nbMachines, j in 1:data.nbBalanceCostData)+sum(data.weightLoadCost[r] * max_value_o1[i,r] for i in 1:data.nbMachines, r in 1:data.nbResources))
    ####################################################
    
    set_optimizer_attribute(m, "logLevel", 0)
    #optimize!(m)    # resolution à l'aide du solveur
    #set_optimizer_attribute(m, "seconds", 300)
    optimize!(m)

    objVal = JuMP.objective_value(m) # to be replaced by the objective value
    soluce = Array{Int64}(undef, data.nbProcess) # create an empty table
    #sum(k*JuMP.value(x[k,i]) for k=1:data.nbMachines)
    for i in 1:data.nbProcess
        soluce[i] = sum(k*round(JuMP.value(x[k,i])) for k=1:data.nbMachines)  # to be replaced by the assignment found by your model
    end
    println(objVal)
    return SolutionGoogle(soluce,objVal)
end

LoadError: syntax: incomplete: "for" at none:93 requires end

### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [21]:
# enter your instance name here 
donnee = "a1_1"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs 
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)


Received instance files model_a1_1.txt and assignment_a1_1.txt
---------Nbre machines----------
4
---------Nbre processus---------
100
--------------------------------
4.4306501e7
Checking the solution 
Solution received is [1, 1, 1, 2, 2, 4, 4, 4, 4, 3, 2, 1, 1, 4, 4, 2, 4, 1, 1, 4, 3, 2, 2, 2, 1, 2, 2, 3, 1, 4, 4, 4, 1, 3, 3, 1, 1, 2, 1, 4, 1, 1, 2, 4, 2, 4, 1, 3, 3, 3, 1, 4, 1, 4, 4, 1, 2, 2, 3, 1, 1, 3, 3, 2, 1, 3, 4, 2, 2, 1, 1, 3, 1, 1, 4, 1, 2, 1, 2, 4, 4, 2, 1, 1, 1, 4, 1, 1, 1, 3, 1, 4, 4, 2, 3, 4, 3, 2, 2, 3]
Cost : 44306501
Checking the size of the vector
Test passed
Checking that the values in the solution are machine indices
Test passed
Checking capacity constraint (without transient)
Test passed
Checking capacity constraint (with transient)
Test passed
Checking disjunctions between processes of the same service
Test passed
Checking spread constraints 
Test passed
Checking dependency constraints
Test passed
All tests passed: solution feasible 
Load cost = 3.101173e7
Balanc